# Vamos aprender a trabalhar com PDF usando o Python

- Regra geral: PDF foi feito justamente para bloquear muita coisa, então não é fácil "brincar" com um pdf
- Mesmo assim, Python tem várias bibliotecas que vão nos ajudar, vamos focar em 2:
    - PyPDF2
    - Tabula
- Ler e extrair informações de um PDF a gente consegue fazer.
- Escrever e Editar, aí já é outra história

### Para os nossos exemplos, vamos avaliar o Release de Resultados do 3º e 4º Trimestre de 2020 da Magazine Luiza

#### 1º Objetivo: Queremos conseguir separar apenas o DRE do Release de Resultados (Página 14) para enviar para a Diretoria, como fazemos?
    - Separar as páginas de um pdf

In [9]:
import PyPDF2 as pyf
from pathlib import Path

pyf.PdfFileReader # para ler um arquivo em PDF
pyf.PdfFileWriter # criar/escrever um arquivo em PDF
pyf.PdfFileMerger # mesclar arquivos PDF

nomeArquivoPdf='MGLU_ER_3T20_POR.pdf'
arquivoPdf=pyf.PdfReader(nomeArquivoPdf)
display(arquivoPdf)

numPagina=1
# torna o arquivo iteravel, o for vai rolar pagina por pagina
for pagina in arquivoPdf.pages: 
    # cria o arquivo com uma pagina apenas
    arquivoNovo=pyf.PdfWriter()
    arquivoNovo.add_page(pagina)
    # salva o arquivo
    # abre o arquivo Arquivo1.pdf em paginas\\ no mode='wb' (modo writable)
    # escreve/salva o arquivo novo no arquivo final
    with Path(f'paginas\\Arquivo{numPagina}.pdf').open(mode='wb') as arquivoFinal:
        arquivoNovo.write(arquivoFinal)
        numPagina+=1

#### 2º Objetivo: Com o Release de Resultados já separado página por página, queremos incluir apenas as Páginas de Destaque (Página 1), DRE (Página 14) e Balanço (Página 16).
    - Juntar vários pdfs em 1

In [10]:
paginasSolicitadas=[1,14,16]

arquivoNovo=pyf.PdfWriter()
for numPagina in paginasSolicitadas:
    # Pegar o arquivo numero da pagina
    # Adicionar no novo PDF
    nomeArquivo=f'paginas\\Arquivo{numPagina}.pdf'
    arquivoPdf=pyf.PdfReader(nomeArquivo)
    pagina=arquivoPdf.pages[0] # tem que ter o indice zero porque por natureza é uma lista de paginas, na ausencia desse passara a lista inteira (mesmo que so haja um elemento)
    arquivoNovo.add_page(pagina)

# salvar o novo PDF
with Path(f'Consolidado.pdf').open(mode='wb') as arquivoFinal:
    arquivoNovo.write(arquivoFinal)

### Extra: Para adicionar todas as páginas de 2 pdfs

In [11]:
pdfMesclado=pyf.PdfMerger()

arquivo1='MGLU_ER_3T20_POR.pdf'
arquivo2='MGLU_ER_4T20_POR.pdf'

# so funciona com um append de cada vez
pdfMesclado.append(arquivo1)
pdfMesclado.append(arquivo2)

with Path(f'ConsolidadoAppend.pdf').open(mode='wb') as arquivoFinal:
    pdfMesclado.write(arquivoFinal)

# Funcionalidades que podem ser úteis:

- Inserir arquivo no meio do outro
- Quero colocar dentro do Resultado do 4T20 os destaques do 3T20 para poder comparar os 2 dentro do mesmo relatório

In [12]:
pdfMesclado=pyf.PdfMerger()

pdfMesclado.append(arquivo1)

# adiciona o arquivo 2 APÓS a pagina número 1 do arquivo 1, nesse caso 1 seria efetivamente a primeira folha e nao 0 como de costume
pdfMesclado.merge(1,arquivo2)

with Path(f'ConsolidadoMerge.pdf').open(mode='wb') as arquivoFinal:
    pdfMesclado.write(arquivoFinal)

- Rodar Página

In [13]:
pdfRodar=pyf.PdfReader(arquivo1)

pdfFinal=pyf.PdfWriter()
for pagina in pdfRodar.pages:
    # 90 deita a pagina, 180 coloca ao contrario
    pagina.rotate(90)
    pdfFinal.add_page(pagina)

with Path(f'Rodado.pdf').open(mode='wb') as arquivoFinal:
    pdfFinal.write(arquivoFinal)

# Trabalhando com Textos e Informações Dentro do PDF

#### 1º Objetivo: Quero identificar como foram as Despesas com Vendas da MGLU
    - Pegar texto da página e identificar onde está essa informação

In [27]:
nomeArquivo='MGLU_ER_3T20_POR.pdf'
arquivo=pyf.PdfReader(nomeArquivo)

# imprime o numero de paginas do arquivo
numPaginas=len(arquivo.pages)
print(numPaginas)

# imprime as informacoes do arquivo
informacoesArquivo=arquivo.metadata
print(informacoesArquivo)


textoProcurado='| Despesas com Vendas'

# percorrendo todas as páginas
numPagina=1
for pagina in arquivo.pages:
    # pegar o que está escrito na página
    textoPagina=pagina.extract_text()
    # textoPagina=pagina.extract_text().replace('  ',' ').replace('\n \n ','&&&&&&').replace('\n','').replace('&&&&&&','\n') # tentativa de padronizar o texto

    # verificar se dentro do texto da pagina tem o texto procurado
    if textoProcurado in textoPagina:
        print(f'Está na página {numPagina}')
        textoFinal=textoPagina

    # se tiver, retornar qual o numero da pagina

    numPagina+=1

# print(textoFinal)

# descobre a posicao do primeiro caractere referente ao texto procurado
posicaoInicialTexto=textoFinal.find('| Despesas com Vendas')
print(posicaoInicialTexto)

# pesquisa a proxima barrinha do texto, o segundo argumento é a partir de que posicao ele vai procurar
posicaoFinalTexto=textoFinal.find('|', posicaoInicialTexto+1)
print(posicaoFinalTexto)

textoDespesa=textoFinal[posicaoInicialTexto:posicaoFinalTexto]
print(textoDespesa)

24
{'/Title': 'DESEMPENHO FINANCEIRO CONSOLIDADO', '/Author': 'an_rezende', '/Subject': 'Receita Bruta', '/Creator': 'Microsoft® Office Word 2007', '/CreationDate': "D:20201109183121-03'00'", '/ModDate': "D:20201109183121-03'00'", '/Producer': 'Microsoft® Office Word 2007'}
Está na página 10
906
1426
| Despesas com Vendas  
 
No 3T20, as despesas com vendas totalizaram R$1.432,6 milhões, equivalentes a 17,2% da receita líquida, 1,1 p.p. menor que no 
3T19 , principalmente devido ao forte crescimento das vendas . Vale ressaltar que a Companhia conseguiu diluir as despesas com 
vendas m esmo investi ndo em maior nível de serviço,  especialmente em  atendimento e logística.  
 
Nos 9M20, as despesas com vendas totalizaram R$3.487,2 milhões, equivalentes a 18,2% da receita líquida (+1,1 p.p. versus  os 
9M19).  
 



#### 2º Objetivo: Quero analisar o DRE (sem ajuste - Página 5)
    - Para ler tabelas em pdf, use o tabula (é ninja)

In [31]:
import tabula.io

# o numero da pagina nao segue a logica dos arrays/listas e comeca por 1
df=tabula.io.read_pdf('MGLU_ER_3T20_POR.pdf', pages=5)
print(df)

# o tabula procura todas as tabelas que estao na pagina, sendo assim ela tem indices, pois cada indice será uma tabela da pagina comecando pelo indice [0], se houver apenas uma sera apenas esse elemento na lista de tabelas
tabela=df[0]

tabela=tabela.dropna(how='all') # deleta todas as LINHAS que estejam com tudo em branco (NaN), utiliza default axis=0 (linha)

tabela=tabela.dropna(how='all',axis=1) # deleta todas as COLUNAS que estejam com tudo em branco (NaN)

tabela.columns=tabela.iloc[0]
tabela=tabela.drop(tabela.index[0])
tabela=tabela.reset_index(drop=True)

display(tabela)

tabela.to_excel('TabelaExtraidaPDF.xlsx')

[    Unnamed: 0                                       Unnamed: 1  Unnamed: 2  \
0          NaN              R$ milhões (exceto quando indicado)         NaN   
1          NaN                                              NaN         NaN   
2          NaN           Vendas Totais1 (incluindo marketplace)         NaN   
3          NaN                                              NaN         NaN   
4          NaN                                    Receita Bruta         NaN   
5          NaN                                  Receita Líquida         NaN   
6          NaN                                              NaN         NaN   
7          NaN                                      Lucro Bruto         NaN   
8          NaN                                     Margem Bruta         NaN   
9          NaN                                              NaN         NaN   
10         NaN                                           EBITDA         NaN   
11         NaN                                    M

,R$ milhões (exceto quando indicado),3T2,3T1,Var(%,9M2,9M1,Var(%
0,Vendas Totais1 (incluindo marketplace),"12.355,5","6.817,6","81,2","28.584,","18.282,6","56,3"
1,Receita Bruta,"10.349,5","5.999,4","72,5","23.652,","16.508,8","43,3"
2,Receita Líquida,"8.308,3","4.864,2","70,8","19.111,","13.501,3","41,6"
3,Lucro Bruto,"2.178,7","1.424,9","52,9","5.034,","3.728,6","35,0"
4,Margem Bruta,"26,2","29,3","-3,1 p","26,3","27,6","-1,3 p"
5,EBITDA,"546,1","501,2","9,0","1.022,","1.276,5","-19,9"
6,Margem EBITDA,"6,6","10,3","-3,7 p","5,3","9,5","-4,2 p"
7,Lucro Líquido,"206,0","235,1","-12,4","172,","753,8","-77,2"
8,Margem Líquida,"2,5","4,8","-2,3 p","0,9","5,6","-4,7 p"
9,Lucro Bruto - Ajustado,"2.178,7","1.488,9","46,3","5.034,","3.964,6","27,0"


#### 3º Objetivo: Quero analisar o Capital de Giro e os Investimentos (ambas as tabelas na página 12)
    - Páginas com mais de 1 tabela

In [33]:
df=tabula.io.read_pdf('MGLU_ER_3T20_POR.pdf', pages=12)

for tabela in df:
    tabela=tabela.dropna(how='all')
    tabela=tabela.dropna(how='all',axis=1)
    display(tabela)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,(+) Contas a Receber (sem Cartões de Crédito),"(26,7)","706,3","680,8","781,3","794,0","733,0"
1,(+) Estoques,"2.120,2","5.005,9","4.198,2","4.075,5","3.801,8","2.885,7"
2,(+) Partes Relacionadas (sem Cartão Luiza),"(10,5)","71,3","80,4","77,1","100,6","81,8"
3,(+) Impostos a Recuperar,"186,3","932,0","748,9","877,4","864,1","745,7"
4,(+) Outros Ativos,"(56,6)","88,5","100,2","143,5","136,3","145,1"
5,(+) Ativos Circulantes Operacionais,"2.212,7","6.804,0","5.808,6","5.954,8","5.696,8","4.591,3"
7,(-) Fornecedores,"2.301,5","6.104,3","5.334,0","4.132,7","5.934,9","3.802,8"
8,(-) Repasses e outros depósitos,"627,3","627,3","639,3","235,9",-,-
9,"(-) Salários, Férias e Encargos Sociais","95,0","444,7","329,0","263,3","354,7","349,8"
10,(-) Impostos a Recolher,"90,7","299,6","206,4","176,9","352,0","208,8"


,R$ milhões,3T20,%,3T19,%.1,Var(%),9M20,%.2,9M20.1,%.3,Var(%).1
1,Lojas Novas,"21,2",14%,"94,6",51%,-78%,"69,0",21%,"121,9",31%,-43%
2,Reformas,"6,1",4%,"8,2",4%,-26%,"14,6",4%,"38,2",10%,-62%
3,Tecnologia,"69,1",45%,"32,6",18%,112%,"147,6",45%,"84,4",22%,75%
4,Logística,"36,3",24%,"32,8",18%,11%,"62,1",19%,"107,4",27%,-42%
5,Outros,"21,5",14%,"17,7",10%,22%,"35,7",11%,"38,8",10%,-8%
6,Total,"154,2",100%,"186,0",100%,-17%,"329,1",100%,"390,7",100%,-16%


#### O que fazer quando o tabula não consegue ler alguma linha da tabela? Como o cabeçalho, no nosso caso?

In [36]:
# primeira tabela da pagina 5 extraida da maneira normal
capitalGiro=df[0]
capitalGiro=capitalGiro.dropna(how='all')
capitalGiro=capitalGiro.dropna(how='all',axis=1)
print(capitalGiro)

# outro metodo para ler a tabela, mais desorganizado porem inclui mais coisas, entao dependendo do caso pode ser util
df2=tabula.io.read_pdf('MGLU_ER_3T20_POR.pdf', pages=12, lattice=True)
tituloCapitalGiro=df2[0]
tituloCapitalGiro=tituloCapitalGiro.dropna(how='all')
tituloCapitalGiro=tituloCapitalGiro.dropna(how='all',axis=1)
tituloCapitalGiro=tituloCapitalGiro.iloc[0]
tituloCapitalGiro=tituloCapitalGiro.dropna()
capitalGiro.columns=tituloCapitalGiro
print(tituloCapitalGiro)

                                       Unnamed: 0 Unnamed: 1 Unnamed: 2  \
0   (+) Contas a Receber (sem Cartões de Crédito)     (26,7)      706,3   
1                                    (+) Estoques    2.120,2    5.005,9   
2      (+) Partes Relacionadas (sem Cartão Luiza)     (10,5)       71,3   
3                        (+) Impostos a Recuperar      186,3      932,0   
4                               (+) Outros Ativos     (56,6)       88,5   
5             (+) Ativos Circulantes Operacionais    2.212,7    6.804,0   
7                                (-) Fornecedores    2.301,5    6.104,3   
8                 (-) Repasses e outros depósitos      627,3      627,3   
9         (-) Salários, Férias e Encargos Sociais       95,0      444,7   
10                        (-) Impostos a Recolher       90,7      299,6   
11                        (-) Partes Relacionadas     (15,8)      109,8   
12                           (-) Receita Diferida      (0,0)       43,0   
13                      (

# Outro método que pode ser útil algum dia: Captar Imagem em um pdf
    - biblioteca pikepdf

In [15]:
from pikepdf import Pdf, PdfImage

filename = "MGLU_ER_3T20_POR.pdf"
example = Pdf.open(filename)

n_arquivo = 0
for i, pagina in enumerate(example.pages):
    for nome, imagem in pagina.images.items():
        imagem_salvar = PdfImage(imagem)
        imagem_salvar.extract_to(fileprefix=f'imgs/Pagina_{i:02}_{n_arquivo}')
        n_arquivo += 1

ModuleNotFoundError: No module named 'pikepdf'

# Substituir texto no pdf tipo contrato

- Não recomendo fazer diretamente pelo Python. Realmente do que vi a melhor opção me parece o Word fazer isso
- Caso precise automatizar, automatize o processo fazendo ele pelo Word
- Quem quiser MUITO fazer isso pelo Python, tem um link aqui que vai te ajudar de uma solução que achei que funciona. Tem seus bugs/cuidados especiais, mas funciona: https://pdf.co/samples/pdf-co-web-api-replace-text-from-pdf-python-replace-text-from-url